In [1]:
import time
import random
import matplotlib.pyplot as plt
import importlib
import taquin
importlib.reload(taquin)
from taquin import *

P = NPuzzle(3)

def bfs(puzzle, dim):
    P.set_dimension(dim)
    P.solve_bfs(puzzle)
    return True

def dfs(puzzle, dim):
    P.set_dimension(dim)
    P.solve_dfs(puzzle)
    return True

def astar(puzzle, dim):
    P.set_dimension(dim)
    P.solve_astar(puzzle)
    return True

def generate_puzzle(size):
    return P.generate(size)

sizes = [2, 3]
methods = [bfs, dfs, astar]
method_names = ["BFS", "DFS", "ASTAR"]
execution_times = {method: [] for method in method_names}

for size in sizes:
    puzzle = generate_puzzle(size)
    for method, name in zip(methods, method_names):
        start_time = time.time()
        method(puzzle, size)
        end_time = time.time()
        execution_times[name].append(end_time - start_time)

for i, size in enumerate(sizes):
    plt.figure()
    times = [execution_times[method][i] for method in method_names]
    plt.bar(method_names, times, color=["skyblue", "lightgreen", "salmon"])
    plt.xlabel("Method")
    plt.ylabel("Execution Time (s)")
    plt.title(f"Execution Times for Puzzle Size {size}x{size}")
    plt.grid(axis='y')
    plt.show()


: 